In [1]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer

from pyspark.sql.functions import col, explode
import pandas as pd
# import findspark 
# findspark.init()  
# import pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
#for later --conf spark.sql.catalogImplementation=hive 

In [3]:
trainSample = spark.read.parquet('train_sample1.parquet')
testSample = spark.read.parquet('test_sample1.parquet')
trainSample.createOrReplaceTempView('trainSample')
testSample.createOrReplaceTempView('testSample')

In [4]:
indexer_obj_1 = StringIndexer(inputCol="user_id", outputCol="user_id_numer").setHandleInvalid("keep")
indexer_model_1 = indexer_obj_1.fit(trainSample)
indexer_df_1 = indexer_model_1.transform(trainSample)

indexer_obj_2 = StringIndexer(inputCol="track_id", outputCol="track_id_numer").setHandleInvalid("keep")
indexer_model_2= indexer_obj_2.fit(indexer_df_1)
indexer_df_2 = indexer_model_2.transform(indexer_df_1)

train_df = indexer_df_2.drop('user_id')
train_df= train_df.drop('track_id')


In [6]:
test_df_1 = indexer_model_1.transform(testSample)
test_df_2= indexer_model_2.transform(test_df_1)

test_df = test_df_2.drop('user_id')
test_df= test_df.drop('track_id')

In [7]:
test_df.show()

+-----+-----------------+-------------+--------------+
|count|__index_level_0__|user_id_numer|track_id_numer|
+-----+-----------------+-------------+--------------+
|    1|               93|     334914.0|        2629.0|
|    1|              259|     334914.0|        1648.0|
|    1|              276|     334914.0|        5546.0|
|    2|              305|     334914.0|        1686.0|
|   30|              322|      99931.0|       84794.0|
|    1|              359|      99933.0|        1766.0|
|    2|              642|     334914.0|       47063.0|
|    1|              701|      99999.0|       14041.0|
|    1|              808|     334914.0|       69616.0|
|    1|              984|     334914.0|       25893.0|
|    1|              988|     334914.0|        1658.0|
|    1|             1058|     334914.0|         447.0|
|    1|             1095|     100063.0|         383.0|
|    5|             1164|     100072.0|       18547.0|
|    1|             1507|     334914.0|       79535.0|
|    4|   

In [8]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="user_id_numer", itemCol="track_id_numer", ratingCol="count",
          coldStartStrategy="drop")
model = als.fit(train_df)
userRecs = model.recommendForAllUsers(10)

# Evaluate the model by computing the RMSE on the test data

# evaluator = RegressionEvaluator(metricName="rmse", labelCol="count",
#                                 predictionCol="prediction")
# rmse = evaluator.evaluate(predictions)

# print("Root-mean-square error = " + str(rmse))

# Generate top 10 song recommendations for each user

### ADJUST FOR 500 SONGS 

# Generate top 10 user recommendations for each song

#songRecs = model.recommendForAllItems(10)

Root-mean-square error = 7.588528461264799


In [10]:
#### THIS WILL BE REMOVED ####

# Generate top 10 movie recommendations for a specified set of users
users = train_df.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)

# Generate top 10 user recommendations for a specified set of movies
# songs = df_dropped.select(als.getItemCol()).distinct().limit(3)
# songSubSetRecs = model.recommendForItemSubset(songs, 10)

In [ ]:
userSubsetRecs.show()

In [11]:
exploded_df = userSubsetRecs.select(userSubsetRecs.user_id_numer,explode(userSubsetRecs.recommendations.track_id_numer))


In [12]:
import pyspark.sql.functions as func
flatten_df = exploded_df.groupby('user_id_numer').agg(func.collect_list('col').alias("col"))

In [14]:
df_dict = flatten_df.collect()
df_dict = [{r['user_id_numer']: r['col']} for r in df_dict]
dict((key,d[key]) for d in df_dict for key in d)

{262469: [50608, 35076, 4576, 5314, 22981, 62245, 54627, 15646, 22906, 104581],
 45953: [54627, 62245, 16999, 50608, 31340, 23388, 25461, 9782, 67491, 15646],
 278371: [50608, 28588, 37291, 5314, 48884, 16901, 46876, 10896, 28633, 95422]}

In [16]:
#test run
predictions.show()

+-----+-----------------+-------------+--------------+------------+
|count|__index_level_0__|user_id_numer|track_id_numer|  prediction|
+-----+-----------------+-------------+--------------+------------+
|    1|           580845|     199743.0|         148.0|  -0.7401133|
|    1|          1104504|     289603.0|        2366.0|   1.1883036|
|    7|           831326|      74503.0|        6397.0|  -1.0921481|
|    1|           766988|      71392.0|       47711.0|    3.418543|
|    2|           182190|     130992.0|       75509.0| -0.29655308|
|    1|           388396|     166437.0|         243.0|  -0.3623914|
|    2|           132892|     122534.0|         897.0|   0.4828575|
|    3|          1220240|     309456.0|        1896.0|   0.6342591|
|    4|           295321|     150190.0|       18218.0|   1.2568204|
|    1|           878538|     250951.0|       20396.0|  0.26326305|
|    1|           105528|     117867.0|       48686.0| -0.31276146|
|    1|            83334|     113915.0|       67

In [19]:
predictions = model.transform(test_df)
predictions.orderBy('prediction', ascending=False).show()

+-----+-----------------+-------------+--------------+----------+
|count|__index_level_0__|user_id_numer|track_id_numer|prediction|
+-----+-----------------+-------------+--------------+----------+
|   30|           170110|     129021.0|        4159.0| 29.096352|
|    2|           659877|     213424.0|       18607.0| 25.425692|
|    1|           157695|      42701.0|        1288.0| 24.202473|
|    5|          1322196|     327200.0|        1302.0| 20.751654|
|    2|          1210818|      92410.0|        2454.0| 14.304222|
|    2|           404513|      54220.0|       11505.0| 13.979452|
|   14|          1356530|     332963.0|        1080.0| 12.923365|
|    1|           943694|     262160.0|        2427.0| 10.379565|
|   11|          1324320|      97854.0|       21793.0|  9.898027|
|    1|             5265|      35694.0|       13421.0|  9.275231|
|    2|          1081617|     285600.0|       12344.0|  8.869305|
|    2|          1125287|     292864.0|        6054.0|  8.486488|
|   26|   

In [24]:
#https://stackoverflow.com/questions/59390481/how-to-implement-ranking-metrics-of-pyspark

from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
metrics = RankingMetrics(predictions)

AttributeError: 'DataFrame' object has no attribute 'ctx'

In [22]:
test = model.recommendForUserSubset(test_df,10)
test.show()

+-------------+--------------------+
|user_id_numer|     recommendations|
+-------------+--------------------+
|       323090|[{4576, 104.34744...|
|       314281|[{11827, 10.25488...|
|       227152|[{10896, 25.36068...|
|       133153|[{50608, 254.1710...|
|        35694|[{50608, 284.8812...|
|        92644|[{25176, 58.62921...|
|       165914|[{11827, 21.78151...|
|       136625|[{16999, 43.67716...|
|       225755|[{50608, 84.67899...|
|       287645|[{48884, 180.0310...|
|       208696|[{28588, 1002.882...|
|       275698|[{11827, 241.7612...|
|       167242|[{11827, 241.0525...|
|       118605|[{1757, 27.664623...|
|       258065|[{54627, 16.34184...|
|        23136|[{50608, 96.51617...|
|       157866|[{9831, 56.422874...|
|       274716|[{48884, 195.6008...|
|       229338|[{1757, 156.13522...|
|       245018|[{4576, 9.095392}...|
+-------------+--------------------+
only showing top 20 rows



Evaluation Metric

In [ ]:
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
scoreAndLabels = predictions.join(ratingsTuple).map(lambda tup: tup[1])
